## ROI Calculation Demo

#### This notebook explains how we implement ROI/Savings calculation per template.
#### The idea is to experiment with CSVs for unified_jobs and events_table and visually verify the calculations using the cluster-job-status-elapsedtime-hostnames table rendered below.
#### To keep things simple, experiment with smaller CSVs.

In [ ]:
!pip install pandas
!pip install termcolor
!pip install IPython
!pip install ipywidgets

In [ ]:
import pandas as pd
from IPython.display import Markdown
from IPython.display import display, HTML as html_print
from ipywidgets import widgets

### Read unified_jobs_table in a Pandas dataframe

In [ ]:
# unified_jobs = pd.read_csv('unified_jobs_fk.csv')
unified_jobs = pd.read_csv('unified_jobs_table.csv')

if unified_jobs.get('cluster_id') is None:
    unified_jobs['cluster_id'] = 1

unified_jobs.head(5)

### Read events_table in a Pandas dataframe

In [ ]:
# events = pd.read_csv('job_events_fk.csv')
events = pd.read_csv('events_table.csv')

if events.get('cluster_id') is None:
    events['cluster_id'] = 1
    
events.head(5)

### Get ALL templates

In [ ]:
templates = sorted(unified_jobs['name'].unique())

### Helper functions

In [ ]:
def get_host_names_for_job(cluster_id, job_id):
    host_names = events[(events['job_id']==job_id) & (events['cluster_id']==cluster_id)]['host_name'].dropna().unique()
    return host_names

In [ ]:
def get_jobs_for_a_template(template):
   job_ids = unified_jobs[unified_jobs['name']==template]['id']
   cluster_ids = unified_jobs[unified_jobs['name']==template]['cluster_id']
   statuses = unified_jobs[unified_jobs['name']==template]['status']
   elapsed_times = unified_jobs[unified_jobs['name']==template]['elapsed']
    
   run_count = 0
   successful_run_count = 0
   failed_run_count = 0

   elapsed_sum = 0
   successful_elapsed_sum = 0
   failed_elapsed_sum = 0
    
   host_run_count = 0
   successful_host_run_count = 0
   failed_host_run_count = 0

   all_rows = []
   derived_stats = {}

   for cluster_id, job_id, status, elapsed in zip(cluster_ids, job_ids, statuses, elapsed_times):
       run_count += 1
       elapsed_sum += elapsed
    
       host_names = get_host_names_for_job(cluster_id, job_id)        
       host_run_count += len(host_names) 
       
#        print(f"{cluster_id} ==> {job_id} ==> {status} ==> {elapsed} ==> {host_names}")            
       
       if status == 'successful':
           successful_run_count += 1
           successful_elapsed_sum += elapsed
           successful_host_run_count += len(host_names)
       elif status == 'failed':
           failed_run_count += 1
           failed_elapsed_sum += elapsed
           failed_host_run_count += len(host_names)
            
       host_run_count_avg = 0 if run_count == 0 else host_run_count / run_count
       successful_host_run_count_avg = 0 if successful_run_count == 0 else successful_host_run_count / successful_run_count
       failed_host_run_count_avg = 0 if failed_run_count == 0 else failed_host_run_count / failed_run_count
    
       row = [cluster_id, job_id, status, elapsed, host_names]
       all_rows.append(row)     
       

   print(f"\nrun_count={run_count}")
   print(f"\nsuccessful_run_count={successful_run_count}")
   print(f"\nfailed_run_count={failed_run_count}")

   print(f"\nelapsed_sum={elapsed_sum}")
   print(f"\nsuccessful_elapsed_sum={successful_elapsed_sum}")
   print(f"\nfailed_elapsed_sum={failed_elapsed_sum}")
    
   print(f"\nhost_run_count={host_run_count}")
   print(f"\nsuccessful_host_run_count={successful_host_run_count}")
   print(f"\nfailed_host_run_count={failed_host_run_count}")

   print(f"\nhost_run_count_avg={host_run_count_avg}")
   print(f"\nsuccessful_host_run_count_avg={successful_host_run_count_avg}")
   print(f"\nfailed_host_run_count_avg={failed_host_run_count_avg}")
    
   print(f"\nknown_roi_metric={successful_run_count * successful_host_run_count_avg}")

   derived_stats['run_count'] = run_count
   derived_stats['successful_run_count'] = successful_run_count
   derived_stats['failed_run_count'] = failed_run_count
    
   derived_stats['elapsed_sum'] = elapsed_sum 
   derived_stats['successful_elapsed_sum'] = successful_elapsed_sum
   derived_stats['failed_elapsed_sum'] = failed_elapsed_sum

   derived_stats['host_run_count'] = host_run_count 
   derived_stats['successful_host_run_count'] = successful_host_run_count
   derived_stats['failed_host_run_count'] = failed_host_run_count
    
   derived_stats['host_run_count_avg'] = host_run_count_avg 
   derived_stats['successful_host_run_count_avg'] = successful_host_run_count_avg
   derived_stats['failed_host_run_count_avg'] = failed_host_run_count_avg 

   derived_stats['known_roi_metric'] = successful_run_count * successful_host_run_count_avg

   return all_rows, derived_stats
    

### Interactive Dropdown to select a template

In [ ]:
from ipywidgets import interact, Dropdown

display (Markdown(f'<h2><span style="color:green;">ROI Demo</span></h2>'))
    
templates_dropdown = Dropdown(options = templates)


@interact(template = templates_dropdown)
def print_template(template):
    marked_down = f'<b><span style="color: black">Template: {template}</span></b>'
    display (Markdown(marked_down))

    all_rows, derived_stats = get_jobs_for_a_template(template)

    df = pd.DataFrame(all_rows, columns=['Cluster ID', 'Job ID','Status', 'Elapsed', 'Hostnames'])
    
    display(html_print(df.to_html()))

    
    @interact(manual_cost=(0, 100.0), automtion_cost=(0, 40.0), manual_time_in_mins=(0, 600.0), style={'text-align': 'left', 'width': '200px'})
    def cost(manual_cost=50.0, automtion_cost=20.0, manual_time_in_mins=60.00):
        print(f"Successful Host Run Count: {derived_stats['successful_host_run_count']}")
        print(f"Successful Elapsed Sum in seconds: {derived_stats['successful_elapsed_sum']}\n")
        
        total_manual_cost = derived_stats['successful_host_run_count'] * float(manual_cost) * float(manual_time_in_mins) / 60
        display (Markdown(f'<h4><span style="color: gray">Total Manual Cost: Successful Host Run Count * (manual_cost * manual_time_in_minutes) = {total_manual_cost}</span></h4>'))
        
        total_automation_cost = round((derived_stats['successful_elapsed_sum']/3600) * float(automtion_cost), 2)
        display (Markdown(f'<h4><span style="color: gray">Total Automation Cost: Successful Elapsed Sum in seconds / 3600 * automation_cost = {total_automation_cost}</span></h4>'))
        
        display (Markdown(f'<h4><span style="color: gray">Total Savings: Total Manual Cost - Total Automation Cost = {total_manual_cost - total_automation_cost}</span></h4>'))
        display (Markdown(f'<h4><span style="color: green">Total Savings for Template {template}</span></h4>{total_manual_cost - total_automation_cost}'))
